In [9]:
import pandas as pd
from scipy import stats
import numpy as np

# --- 1. Load Data and Define Segments ---

# Load your dataset with the CORRECT path and separator
df = pd.read_csv("../data/MachineLearningRating_v3.txt", sep="|", low_memory=False)

# --- Define Segments for T-Tests ---
# For Margin and Claim Severity T-tests, we compare two groups. 
# We'll compare the two most frequent Postal Codes to ensure enough samples.
top_postal_codes = df['PostalCode'].value_counts().nlargest(2).index.tolist()

if len(top_postal_codes) >= 2:
    POSTAL_CODE_A = top_postal_codes[0]
    POSTAL_CODE_B = top_postal_codes[1]
else:
    # Fallback/Error case if data is too small
    POSTAL_CODE_A = None
    POSTAL_CODE_B = None
    print("Warning: Insufficient unique Postal Codes for A/B segmentation.")

# --- 2. Metric Creation ---

# Margin: TotalPremium - TotalClaims
df['Margin'] = df['TotalPremium'] - df['TotalClaims']

# Claim Frequency: 1 if claim > 0, 0 otherwise
df['HasClaim'] = (df['TotalClaims'] > 0).astype(int)

# Claim Severity: Claim amount given TotalClaims > 0.
# Create a new DataFrame for severity analysis (only policies with claims)
# This is used for ANOVA/T-tests on the *amount* of the claim.
df_severity = df[df['TotalClaims'] > 0].copy()


# --- 3. Hypothesis Testing Functions ---

# Helper function to generate standardized conclusion
def get_conclusion(p_value):
    if p_value < 0.05:
        return f"Reject H₀ (p={p_value:.4f}): Significant difference exists."
    else:
        return f"Fail to Reject H₀ (p={p_value:.4f}): No significant difference found."

# H₀: No risk (frequency) differences across provinces (Chi-Squared)
def test_risk_frequency_province(df):
    try:
        chi2, p, _, _ = stats.chi2_contingency(pd.crosstab(df['Province'], df['HasClaim']))
        return f"Province Frequency Test: {get_conclusion(p)}"
    except ValueError:
        return "Province Frequency Test: Failed (Small Sample Warning/Sparse Data). Results unreliable."

# H₀: No risk (severity) differences across provinces (ANOVA)
def test_risk_severity_province(df_severity):
    try:
        # ANOVA requires lists of claim amounts for each province with claims
        province_groups = [
            df_severity['TotalClaims'].loc[df_severity['Province'] == p].values 
            for p in df_severity['Province'].unique()
            if len(df_severity['TotalClaims'].loc[df_severity['Province'] == p]) >= 2
        ]
        
        # Check if we have enough groups to perform ANOVA
        if len(province_groups) < 2:
             return "Province Severity Test: Failed (Not enough provinces with claims for ANOVA)."

        f_stat, p = stats.f_oneway(*province_groups)
        return f"Province Severity (ANOVA) Test: {get_conclusion(p)}"
    except Exception as e:
        return f"Province Severity (ANOVA) Test: Failed due to error: {e}"

# H₀: No risk (frequency) differences between zip codes A and B (Chi-Squared)
def test_risk_frequency_zip_codes_ab(df, code_a, code_b):
    if not code_a or not code_b:
        return "Zip Code Frequency Test: Skipped due to insufficient postal codes."
        
    df_ab = df[df['PostalCode'].isin([code_a, code_b])]
    contingency_table = pd.crosstab(df_ab['PostalCode'], df_ab['HasClaim'])
    
    try:
        chi2, p, _, _ = stats.chi2_contingency(contingency_table)
        return f"Zip Code (A vs B) Frequency Test: {get_conclusion(p)}"
    except ValueError:
        return "Zip Code (A vs B) Frequency Test: Failed (Small Sample Warning/Sparse Data)."

# H₀: No significant margin difference between zip codes A and B (T-test)
def test_margin_difference_ab(df, code_a, code_b):
    if not code_a or not code_b:
        return "Zip Code Margin Test: Skipped due to insufficient postal codes."
        
    group_a = df.loc[df['PostalCode'] == code_a, 'Margin']
    group_b = df.loc[df['PostalCode'] == code_b, 'Margin']
    
    # Ensure both groups have samples to prevent T-test crash
    if len(group_a) < 2 or len(group_b) < 2:
        return "Zip Code Margin Test: Failed (Insufficient samples in selected A/B groups)."
        
    t_stat, p = stats.ttest_ind(group_a, group_b, equal_var=False) # Unequal variance T-test
    return f"Zip Code Margin Test (A vs B): {get_conclusion(p)}"

# H₀: No risk (frequency) difference between Women and Men (Chi-Squared)
def test_risk_frequency_gender(df):
    try:
        chi2, p, _, _ = stats.chi2_contingency(pd.crosstab(df['Gender'], df['HasClaim']))
        return f"Gender Frequency Test: {get_conclusion(p)}"
    except ValueError:
        return "Gender Frequency Test: Failed (Small Sample Warning/Sparse Data)."

# H₀: No risk (severity) difference between Women and Men (T-test)
def test_risk_severity_gender(df_severity):
    group_men = df_severity.loc[df_severity['Gender'] == 'M', 'TotalClaims']
    group_women = df_severity.loc[df_severity['Gender'] == 'F', 'TotalClaims']
    
    # Ensure both groups have samples
    if len(group_men) < 2 or len(group_women) < 2:
        return "Gender Severity Test: Failed (Insufficient samples in M/F claim groups)."

    t_stat, p = stats.ttest_ind(group_men, group_women, equal_var=False) # Unequal variance T-test
    return f"Gender Severity (T-test) Test: {get_conclusion(p)}"


# --- 4. Execute Tests and Print Results ---

print("## 📊 Task 3: Hypothesis Testing Results\n")
print(f"**Postal Code Segments Used:** A='{POSTAL_CODE_A}', B='{POSTAL_CODE_B}'\n")
print("---")
print("### 📍 Risk Differences by Province")
print(test_risk_frequency_province(df))
print(test_risk_severity_province(df_severity))
print("---")
print("### 🗺️ Risk and Margin Differences by Postal Code (A vs B)")
print(test_risk_frequency_zip_codes_ab(df, POSTAL_CODE_A, POSTAL_CODE_B))
print(test_margin_difference_ab(df, POSTAL_CODE_A, POSTAL_CODE_B))
print("---")
print("### 🚻 Risk Differences by Gender")
print(test_risk_frequency_gender(df))
print(test_risk_severity_gender(df_severity))
print("---")

## 📊 Task 3: Hypothesis Testing Results

**Postal Code Segments Used:** A='2000', B='122'

---
### 📍 Risk Differences by Province
Province Frequency Test: Reject H₀ (p=0.0000): Significant difference exists.
Province Severity (ANOVA) Test: Reject H₀ (p=0.0000): Significant difference exists.
---
### 🗺️ Risk and Margin Differences by Postal Code (A vs B)
Zip Code (A vs B) Frequency Test: Fail to Reject H₀ (p=0.0579): No significant difference found.
Zip Code Margin Test (A vs B): Fail to Reject H₀ (p=0.2445): No significant difference found.
---
### 🚻 Risk Differences by Gender
Gender Frequency Test: Reject H₀ (p=0.0266): Significant difference exists.
Gender Severity Test: Failed (Insufficient samples in M/F claim groups).
---


Null Hypothesis (H0​),Metric Tested,Test Used,P-value,Conclusion
H₀: No risk difference across provinces,Claim Frequency,χ2,0.0000,Reject H₀
H₀: No risk difference across provinces,Claim Severity,ANOVA,0.0000,Reject H₀
H₀: No risk difference between zip codes (A vs B),Claim Frequency,χ2,0.0579,Fail to Reject H₀
H₀: No margin difference between zip codes (A vs B),Margin,T-test,0.2445,Fail to Reject H₀
H₀: No risk difference between Women and Men,Claim Frequency,χ2,0.0266,Reject H₀
H₀: No risk difference between Women and Men,Claim Severity,T-test,N/A,Test Failed